In [1]:
import os
import torch
from ultralytics import RTDETR
import yaml
import sys
from pathlib import Path

ROOT_DIR = Path().resolve().parents[1]
sys.path.append(str(ROOT_DIR))
from src.utils.config_loader import load_config
cfg = load_config()


In [2]:
dataset_yaml = ROOT_DIR / cfg["paths"]["splits_path"] / "dataset.yaml"

In [ ]:
# Definición de la ruta para modelos base
base_models_dir = ROOT_DIR / cfg["paths"]["models_base"]
base_models_dir.mkdir(parents=True, exist_ok=True) # Crea la carpeta si no existe

# Ruta completa al archivo .pt
model_path = base_models_dir / "rtdetr-l.pt"

In [4]:
model = RTDETR(model_path)

# Entrenamiento 

In [6]:
# Iniciar el proceso de entrenamiento [cite: 482, 555]
results = model.train(
    data=dataset_yaml,
    epochs=cfg['training']['epochs'],       # 50 épocas según tus logs
    batch=cfg['training']['batch_size'],    # Ajustado a la capacidad de la RTX 4090
    imgsz=640,
    project=cfg["rtdetr_train"],
    name="rtdetr-l",
    device=0 if torch.cuda.is_available() else "cpu",
    pretrained=True,
    plots=True, # Generar automáticamente gráficas de entrenamiento
    exist_ok=True
)


New https://pypi.org/project/ultralytics/8.4.14 available  Update with 'pip install -U ultralytics'
Ultralytics 8.4.11  Python-3.12.3 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Bureau\Proyectos\DATAGIA\Modelo-CU11\data\splits\dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=C:\Bureau\Proyectos\DATAGIA\Model

KeyboardInterrupt: 

In [ ]:
# Validación final de RT-DETR
metrics = model.val()
print(f"mAP50 Global RT-DETR: {metrics.box.map50:.4f}")